# Implementação do Algoritmo K-Nearest Neighbors (kNN)

Este notebook contém a implementação do algoritmo k-Nearest Neighbors (kNN) do zero, utilizando apenas a biblioteca NumPy, além de uma análise crítica do código gerado e dos resultados obtidos ao aplicá-lo no dataset Penguins.

## Prompts Utilizados e Respostas

### Prompt 1:

Implemente o algoritmo K-Nearest Neighbors (kNN) from scratch utilizando apenas a biblioteca NumPy para operações vetoriais. A implementação não pode utilizar outra bibliotecas prontas, o teste do algoritmo deve ser feito no dataset penguins_lter.csv na pasta raiz do projeto. Após a implementação e execução do algoritmo, printe na tela o desempenho do modelo utilizando as seguintes métricas:
Matriz de confusão
Acurácia
Recall
Precisão
F1-score

### Resposta 1:

Aqui está a implementação do algoritmo K-Nearest Neighbors (kNN) do zero, utilizando apenas a biblioteca NumPy. A implementação será feita na classe knn.py, e será utilizado o dataset Penguins para testar o algoritmo. As métricas solicitadas (matriz de confusão, acurácia, recall, precisão e F1-score) também serão calculadas.

In [43]:
import numpy as np
from collections import Counter
import csv

In [44]:
class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def predict(self, X_test):
        predictions = [self._predict(x) for x in X_test]
        return np.array(predictions)

    def _predict(self, x):
        # Calcula a distância entre x e todos os pontos de treino
        distances = [self.euclidean_distance(x, x_train) for x_train in self.X_train]
        # Obtem os k vizinhos mais próximos
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        # Retorna a classe mais comum entre os vizinhos
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

In [45]:
def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

In [46]:
def confusion_matrix(y_true, y_pred):
    unique_labels = np.unique(y_true)
    matrix = np.zeros((len(unique_labels), len(unique_labels)), dtype=int)
    label_to_index = {label: index for index, label in enumerate(unique_labels)}
    
    for true, pred in zip(y_true, y_pred):
        matrix[label_to_index[true], label_to_index[pred]] += 1
    
    return matrix

In [47]:
def precision_recall_f1(conf_matrix):
    TP = np.diag(conf_matrix)
    FP = np.sum(conf_matrix, axis=0) - TP
    FN = np.sum(conf_matrix, axis=1) - TP

    # Precisão
    precision = np.mean(np.divide(TP, (TP + FP), where=(TP + FP) != 0))
    # Recall
    recall = np.mean(np.divide(TP, (TP + FN), where=(TP + FN) != 0))
    # F1-Score
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    
    return precision, recall, f1

In [48]:
def load_penguin_dataset(filename='penguins_lter.csv'):
    X = []
    y = []
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Pular o cabeçalho
        for row in reader:
            try:
                # Selecionar as colunas numéricas: Comprimento do Bico, Profundidade do Bico, Comprimento das Nadadeiras, Massa Corporal
                features = [float(row[9]), float(row[10]), float(row[11]), float(row[12])]
                species = row[2]  # Usamos "Species" como a classe
                if '' not in row[9:13]:  # Verifica se não há valores ausentes
                    X.append(features)
                    y.append(species)
            except ValueError:
                # Ignorar linhas com valores inválidos
                continue
    return np.array(X), np.array(y)

In [49]:
if __name__ == "__main__":
    # Carregar dataset de pinguins
    X, y = load_penguin_dataset()

    # Embaralhar o dataset
    indices = np.random.permutation(len(X))
    X = X[indices]
    y = y[indices]

    # Dividir em treino e teste (80% treino, 20% teste)
    split_ratio = 0.8
    split_index = int(split_ratio * len(X))
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # Instanciar o modelo
    knn = KNN(k=3)
    knn.fit(X_train, y_train)

    # Fazer previsões
    y_pred = knn.predict(X_test)

    # Calcular a matriz de confusão
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("Matriz de Confusão:")
    print(conf_matrix)

    # Calcular acurácia
    acc = accuracy(y_test, y_pred)
    print(f"Acurácia: {acc:.4f}")

    # Calcular precisão, recall e F1-score
    precision, recall, f1 = precision_recall_f1(conf_matrix)
    print(f"Precisão: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")


Matriz de Confusão:
[[20  2  3]
 [ 6 10  1]
 [ 2  1 24]]
Acurácia: 0.7826
Precisão: 0.7802
Recall: 0.7590
F1-Score: 0.7695


### Explicação do código:

1. *Classe KNN*:
   - __init__: Define o valor de k, o número de vizinhos.
   - fit: Armazena os dados de treino.
   - euclidean_distance: Calcula a distância euclidiana.
   - predict: Faz a previsão para o conjunto de dados de teste.

2. *Funções de Métricas*:
   - accuracy: Calcula a acurácia.
   - confusion_matrix: Gera a matriz de confusão.
   - precision_recall_f1: Calcula precisão, recall e F1-score.

3. *Carregamento do Dataset*:
   - load_penguin_dataset: Lê o arquivo penguins_lter.csv e retorna o conjunto de características (X) e as classes (y).

4. *Execução do Algoritmo*:
   - O dataset é embaralhado aleatoriamente e dividido em 80% para treino e 20% para teste.
   - O modelo kNN é treinado e testado.
   - As métricas são calculadas e exibidas.

### Exemplo de Saída:

Matriz de Confusão:<br>
[[20  2  3]<br>
 [ 6 10  1]<br>
 [ 2  1 24]]<br>
Acurácia: 0.7826<br>
Precisão: 0.7802<br>
Recall: 0.7590<br>
F1-Score: 0.7695

## Análise Crítica do Código Gerado

Testamos o algoritmo KNN para dois datasets diferentes: Iris e Penguins, sendo Penguins o dataset escolhido para a implementação do código.
Escolhendo o dataset Iris, vimos que os resultados de métrica são todos muito próximos de 1.0000 ou exatamente 1.0000.
Isso pode ter ocasionado por diversos fatores:
- *Valor de k muito baixo*: Ocasionando overfitting.
- *Tamanho do dataset muito reduzido*
- *Falta de cross-validation*

Por conta desses fatores, resolvermos optar pelo dataset Penguins, pois temos dados mais realistas, mais bem distribuidos e mais casos de teste.
<br><br><br>
Em relação ao dataset Penguis:

#### Pontos Fortes
- *Implementação simples*: O código é direto e fácil de entender, que utiliza o algoritmo kNN.

#### Potenciais Problemas/Diferenças
- *Divisão Simples de Dados*: A divisão do dataset em treino e teste é feita de forma aleatória e fixa (80/20), sem cross-validation, assim não fazendo a melhor divisão dos dados.
- *Escolha de k*: Com o valor de k muito baixo ocorre casos de overfitting e com o valor de k muito alto ocorre casos de underfitting.

#### Opções de Projeto e Impacto
- *Uso de Distância Euclidiana*: Simples e eficaz para um dataset com dados equilibrados, mas com sensibilidade a outliers.

#### Sugestões de Melhorias
- *Cross-validation*: Implementar cross-validation para uma melhora divisão dos dados.
- *Normalização dos Dados*: Melhora a performance de dados com diferentes escalas.
- *Otimização de k*: Testar diferentes valores de k para encontrar o melhor valor para o dataset.


### Conclusão:
Primeiramente, testamos com um dataset pequeno e simples (Iris) e, com essa implementação de algoritmo, obtemos resultados perfeitos, resultados irreais.
Por isso optamos por usar o dataset Penguins, que nos mostrou resultados reais, porém com um desempenho não tão satisfatório.
Podemos melhorar o algoritmo implementando cross-validation, realizando normalização de dados e achando o valor ideal para k.